# Problems and Comments 
---

### There are a few problems with this program and with the model itself (that can be solved): 

1. Estimation of Covariance matrix is very messy, .cov() is a rather inaccurate estimation of the covariance matrix 

2. Exp. Returns (Mean Returns) is a quite bad criterion for futurue returns, Markowitz himself said that

3. Mus vector (exponentially distributed), is rather inaccurate at finding the 'true' convex region. The mus vector might be a tricky thing to solve; it is the parameter that the model solves for a given variance - so as of now this is a exponentially distributed scale, which is later multiplied by the covariance matrix in the optimization (see: https://cvxopt.org/examples/book/portfolio.html). More intuitively (and closer to the actual model), mus should be distributed between the minimum return (either simply 0, or the return of the minimum-return asset) and the maximum return asset. The problem is, that the constraint matrices will have to be modified accordingly (instead of multiplying mu by covariance)

4. This is *the* foundational model in portfolio optimization, better and more robust models have been suggested

5. Vizualisation of the portfolio of the results is not great as of now, there are many cool things that can be done (@Ash), simply way would be to plot the efficient frontier (like in the picture below), play with visualization of portfolios (aka pie charts, doughnut charts, stacked bar chart (to show distriubtion with different risk) or tree maps)

![Tangent Portfolio](https://github.com/kgeoffrey/Mean-Variance-Model/blob/master/Notebooks/tangentportfolio.gif?raw=true)

---

### Some solutions I thought of 

1. The math here can get *hard*, luckily there are some libraries that can help with estimating a robust covariance matrix (http://scikit-learn.org/stable/modules/covariance.html)

2. This can be done by simply setting all returns to 1 or using log returns instead! (there is a pitfall though: https://papers.ssrn.com/sol3/papers.cfm?abstract_id=1549328) 

3. Tricky and probabaly the most work, a quantopian user managed to do this (https://www.quantopian.com/posts/the-efficient-frontier-markowitz-portfolio-optimization-in-python-using-cvxopt). The program as of now can allow for short selling and put restrictions on invidual assets weights,by modifying h and G matrices accordingly. I will have to figure out new ways of implementing these features after using the better formulation of the optimization probelm

4. The Mean-Absolute-Deviation Model is finished and ready for upload (it is not parametrized however, it only solves for the minimum variance portfolio - also the region is not convex, a story for another time). Interesting to implement would be the Black-Litterman Model (as it allows to incorporate one's own views; aka I think security x will outperfom security y), there are other models but they are very difficult to implement (downside risk models etc.)

5. probably coolest and easiest to do rn, viz of results via matplotlib and also interactive via bokeh. I do have a very cool idea for a visualization of the process of finding 'optimal portfolios', by running a montecarlo optimization (random plotting to show how the frontier emerges) and saving it as a GIF (imagine the karma :D). Here is a reddit link that explains how to do this with in python for approximating e: https://www.reddit.com/r/dataisbeautiful/comments/91rgzy/monte_carlo_simulation_of_e_oc/e3046ph/, also see below for reference of what it could look like!

![Montecarlo Optimization](https://github.com/kgeoffrey/Mean-Variance-Model/blob/master/Notebooks/ezgif.com-video-to-gif.gif?raw=true)

Apparently gif is too large for github, but it will display in your notebook! 

In [63]:
## Put libraries to be used here

# libs for solving problem
import decimal
import pandas as pd
import numpy as np
import cvxopt as opt
from cvxopt import blas, solvers
import quandl
# API configuration here
quandl.ApiConfig.api_key = "VAA5bZ67DimoDkvMStuG"
solvers.options['show_progress'] = False


# libs for visualization
import matplotlib.pyplot as plt

from bokeh.plotting import figure
from bokeh.io import show, output_notebook, output_file, save
from bokeh.models import ColumnDataSource, HoverTool, CheckboxGroup, Panel
from bokeh.models.widgets import RangeSlider, Slider, Tabs
from bokeh.palettes import Category10_5, Category20_16
from bokeh.layouts import column, row, WidgetBox
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application

## 3. Below I will try to figure out how to reformulate the problem as described in 3.

Need to construct new h and G matrices to adjust return

In [64]:
## This is the correct implementatio by the quantopian user

def optimal_portfolio(returns):
    n = len(returns)
    returns = np.asmatrix(returns)
    
   
    # Convert to cvxopt matrices
    S = opt.matrix(np.cov(returns))
    pbar = opt.matrix(np.mean(returns, axis=1))

    # Determine trial mus based on observed returns
    N=25
    mus_min=max(min(pbar),0)
    mus_max=max(pbar)
    mus_step=(mus_max - mus_min) / (N-1)
    mus = [mus_min + i*mus_step for i in range(N)]
    
    
    # print mus

    # Create constraint matrices
    G=opt.matrix(np.concatenate((-np.transpose(pbar),-np.identity(n)),0))
    A = opt.matrix(1.0, (1, n))
    b = opt.matrix(1.0)
    
    # Calculate efficient frontier weights using quadratic programming
    portfolios=[]
    for r_min in mus:
       h=opt.matrix(np.concatenate((-np.ones((1,1))*r_min,np.zeros((n,1))),0))
       sol = solvers.qp(S, -pbar, G, h, A, b)['x']
       
       portfolios.append(sol)
    
    ## CALCULATE RISKS AND RETURNS FOR FRONTIER
    returns = [blas.dot(pbar, x) for x in portfolios]
    risks = [np.sqrt(blas.dot(x, S*x)) for x in portfolios]
    ## CALCULATE THE 2ND DEGREE POLYNOMIAL OF THE FRONTIER CURVE
    m1 = np.polyfit(returns, risks, 2)
    x1 = np.sqrt(m1[2] / m1[0])
    # CALCULATE THE OPTIMAL PORTFOLIO
    h=opt.matrix(np.concatenate((-np.ones((1,1))*x1,np.zeros((n,1))),0))
    wt = solvers.qp(S, -pbar, G, h, A, b)['x']
    return np.asarray(wt)

In [83]:
pbar = np.mean(returns, axis=1) 
n = len(returns.T)
r_min = np.min(pbar)

h=(np.concatenate((-np.ones((1,1))*r_min,np.zeros((n,1))),0))
#G=np.concatenate((-np.transpose(pbar),-np.identity(n)),0)

#print(h)
#print(n)

print(len(-np.transpose(pbar)))

1


In [93]:
n = len(returns.T)
returns = np.asmatrix(returns.T)
pbar = opt.matrix(np.mean(returns, axis=1))
G=opt.matrix(np.concatenate((-np.transpose(pbar),-np.identity(n)),0))

print(len(G))
print(len(h))

210
20


# 5. Visualization 

Below we can work on 5. visulatization. I simply posted the code from the other notebook (without the pie-charts, doughnut chart or risk-return plots). Also simplified the code, so it is more readable, simply run the cell and play with the output below! 

In [68]:
## 1. List of stock tickers 

stocks1 = ['AMD', 'NVDA', 'OKE', 'CHK', 'NEM', 'AMAT', 'ALB', 'FCX', 'HPE', 'IDXX', 'WMB']
stocks2 = ['TIVO', 'JCP', 'F']

## 2. Getting Data for the stock tickers 

def datafunction(tickers):
    data = quandl.get_table('WIKI/PRICES', ticker = tickers, 
                            qopts = { 'columns': ['ticker', 'date', 'adj_close'] },
                            date = { 'gte': '2017-11-31', 'lte': '2017-12-31' }, 
                            paginate=True)
    new = data.set_index('date')
    # use pandas pivot function to sort adj_close by tickers
    clean_data = new.pivot(columns='ticker')
    return clean_data

stocklist = stocks1 + stocks2
clean_data = datafunction(stocklist)

## 3. Parameters (don't change for now, you may change Scale to plot more portfolios!)

Minimal_gewicht1 = -0.0
Maximal_gewicht1 = 0.1
Minimal_gewicht2 = 0.0
Maximal_gewicht2 = 0.1
Scale = 100 
Riskaversion = 99 
rf = 0.01 

## 4. Transforming Data for optimization 

returnss = (clean_data.pct_change().dropna())
number = len(stocklist)
returns = returnss.values #as_matrix()

## 5. Optimization! various outputs will explain in the cell below! 

def optimal_portfolio(returns):
    
    n = len(returns)
    N1 = len(stocks1)
    N2 = len(stocks2)
    returns = np.asmatrix(returns)
    
    G = Scale
    mus = [10**(5.0 * t/G - 1.0) for t in range(G)]

    S = opt.matrix(np.cov(returns))
    pbar = opt.matrix(np.mean(returns, axis=1))
    
    dank = -np.array(np.eye(n))
    dabbie = np.array(np.eye(n))
    ye = np.vstack((dank, dabbie))
    G = opt.matrix(ye, tc='d')
    
    d1 = -np.ones((N1,1))*Minimal_gewicht1 
    e1 = np.ones((N1,1))*Maximal_gewicht1
    d2 = -np.ones((N2,1))*Minimal_gewicht2
    e2 =  np.ones((N2,1))*Maximal_gewicht2
    min_constraint = np.vstack((d1, d2))
    max_constraint = np.vstack((e1, e2))

    dodo = np.vstack((min_constraint, max_constraint))
    h = opt.matrix(dodo, tc='d')
    A = opt.matrix(1.0, (1, n))
    b = opt.matrix(1.0)

    
    # Calculate efficient frontier weights
    portfolios = [solvers.qp(mu*S, -pbar, G, h, A, b)['x'] for mu in mus]
    
    # CALCULATE RISKS AND RETURNS FOR FRONTIER  (will add back later)
    returns = np.asarray([blas.dot(pbar, x) for x in portfolios])
    risks = np.asarray([np.sqrt(blas.dot(x, S*x)) for x in portfolios])
    
    # Maximum Return Portfolio (will always return weights with 100% in asset with maximum mean return)
    m1 = np.polyfit(returns, risks, 2)
    x1 = np.sqrt((m1[2])/ m1[0])
    maxr_opt = solvers.qp(opt.matrix(x1 * S), -pbar, G, h, A, b)['x']
    
    # approximates the position of the maximum Sharpe portfolio (also called tangent portfolio)
    # the bigger the scale parameter the more accurate the calculation of the maximum sharpe portfolio!
    slope = (returns-(rf/252))/risks
    sharpe_opt = slope.argmax()
    
    # Minimum Variance Portfolio
    mini_var =  np.array(mus).max()
    minv_opt = solvers.qp(opt.matrix(mini_var * S), -pbar, G, h, A, b)['x']
    
    return np.asarray(maxr_opt), returns, risks, portfolios, sharpe_opt, mini_var

max_return_weights, exp_returns, exp_risk, weights, max_sharpe_weights, min_var_weights = optimal_portfolio(returns.T)


#index portfolios on the efficient frontier here
test1=np.array(opt.matrix(weights))
sublist=[test1[n:n+number] for n in range(0,len(test1),number)]
real=sublist[Riskaversion]

In [69]:
## You can play with the output here! simply run all the code above! 

# the variable sublist, containst a list (of 100? --> check scale variable) of portfolios on the optimal boundary

#This prints the maximum sharpe portfolio for instance! 
print(sublist[max_sharpe_weights])



[[9.99997480e-02]
 [9.99997806e-02]
 [2.00707982e-07]
 [1.00742379e-02]
 [9.99999299e-02]
 [9.99999865e-02]
 [9.99998989e-02]
 [9.99999553e-02]
 [1.87472849e-07]
 [9.99999480e-02]
 [7.22147692e-02]
 [9.99999635e-02]
 [1.77114262e-02]
 [9.99999677e-02]]
